In [7]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
llm = ChatOpenAI(
    temperature=0.6,
    model="glm-4.6",
    openai_api_key=os.getenv("ZAI_API_KEY"),
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)


In [10]:
#提取信息
prompt_extract = ChatPromptTemplate.from_template(
    "请从以下文本中提取技术规格：\n\n{text_input} "
)

#转为json
prompt_transform = ChatPromptTemplate.from_template(
    "请将以下技术规格转为 JSON 格式，包含 'cpu'、'memory' 和 'storage' 三个键：\n\n{specifications}"
)

#构建链
#stroutputParser()将消息转为字符串
extraction_chain = prompt_extract | llm | StrOutputParser()

#将消息提供为specification

full_chain = (
    {"specifications":extraction_chain}
    | prompt_transform
    | llm
    | StrOutputParser()
)

#执行链条
input_txt = "新款笔记本配备 3.5GHz 八核处理器、16GB 内存和 1TB NVMe SSD"

final_result = full_chain.invoke({"text_input":input_txt})

print("json输出：",final_result)

json输出： 
好的，这是根据您提供的技术规格转换成的 JSON 格式。

我使用了嵌套对象来组织每个部分的详细属性，这是一种清晰且可扩展的结构。

```json
{
  "cpu": {
    "frequency": "3.5GHz",
    "cores": "八核"
  },
  "memory": {
    "capacity": "16GB"
  },
  "storage": {
    "type": "NVMe SSD",
    "capacity": "1TB"
  }
}
```

### 格式说明：

*   **顶层键**: `cpu`, `memory`, `storage` 符合您的要求。
*   **嵌套对象**: 对于包含多个属性的 `cpu` 和 `storage`，我使用了嵌套对象来分别存储其详细信息（如频率、核心数、类型、容量）。
*   **键名**: 我将中文属性名翻译成了更通用的英文键名（例如，`频率` -> `frequency`），这是在 JSON 中的标准做法。
*   **值**: 所有值都以字符串形式保留，以忠实于原始文本（例如，"3.5GHz", "八核", "1TB"）。
